In [1]:
import json
import numpy as np
import os
import pandas as pd
import time

cwd = os.getcwd()
pkg_dir = os.path.dirname(cwd)
data_dir = os.path.join(pkg_dir, 'tests/data')

from reV.handlers.transmission import TransmissionFeatures
from reV.supply_curve.supply_curve import SupplyCurve

In [15]:
path = os.path.join(data_dir, 'sc_tables', '2_ri_agg_points_cache_064_sj_infsink.csv')
sc_table = pd.read_csv(path)

sc_gids = sc_table[['sc_point_gid', 'sc_point_row_id', 'sc_point_col_id']]
sc_gids = sc_gids.rename(columns={'sc_point_gid': 'gid',
                                  'sc_point_row_id': 'sc_row_ind',
                                  'sc_point_col_id': 'sc_col_ind'})

path = os.path.join(data_dir, 'sc_tables', '5_ri_subset_rural_pv_conus_multiyear.csv')
sc_results = pd.read_csv(path)
sc_results = sc_results.merge(sc_gids, on='gid')

trans_feats = TransmissionFeatures(sc_table)

path = os.path.join(data_dir, 'sc_out', 'baseline_agg_summary.csv')
sc_points = pd.read_csv(path).rename(columns={'sc_gid': 'sc_point_gid'})
sc_points['sc_gid'] = sc_points.index

sc = SupplyCurve(sc_points, sc_table)
sc._serial_sort()
serial_final = sc._sc_points.copy()

In [3]:
# Full sort
sc = SupplyCurve(sc_points, sc_table)
ts = time.time()
sc._serial_sort(connectable=False)
print('Full sort = {:.4f} seconds'.format(time.time() - ts))

# Only connectable
sc = SupplyCurve(sc_points, sc_table)
ts = time.time()
sc._serial_sort()
print('Only connectable {:.4f} seconds'.format(time.time() - ts))

Full sort = 3.4601 seconds
Only connectable 2.5374 seconds


In [12]:
sc._sc_points.head()

,area_sq_km,capacity,gen_gids,gid_counts,latitude,longitude,mean_cf,mean_lcoe,mean_res,pct_slope,res_class,res_gids,sc_col_ind,sc_point_gid,sc_row_ind,trans_gid,trans_type,lcot
sc_gid,,,,,,,,,,,,,,,,,,
0,2.3409,84.2724,[7],[289],41.993,-71.807999,0.124,NaN,3.991000,3.195715,0,[1316631],1,1,0,43451.0,TransLine,1.944053e+06
1,1.8954,68.2344,[7],[234],41.993,-71.737999,0.124,NaN,3.991000,3.121439,0,[1316631],2,2,0,68810.0,Substation,8.856920e+05
2,8.0595,290.1420,"[22, 40]","[719, 276]",41.993,-71.737999,0.124,NaN,4.005335,2.845650,1,"[1317063, 1318507]",2,2,0,80843.0,PCALoadCen,2.343353e+07
3,4.8438,174.3768,[66],[598],41.993,-71.668999,0.124,NaN,3.996000,2.908158,0,[1320698],3,3,0,68810.0,Substation,7.536610e+05
4,3.3696,121.3056,"[40, 76]","[356, 60]",41.993,-71.668999,0.124,NaN,4.003163,2.859689,1,"[1318507, 1321872]",3,3,0,68810.0,Substation,5.242859e+05
